# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [7]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

- Import libraries, APIs and LLM

In [8]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [9]:
import os
import sys

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv
from utils import get_openai_api_key, get_serper_api_key

_ = load_dotenv(find_dotenv())  # read local .env file

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [10]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [11]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
    "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
    "uses statistical modeling and machine learning "
    "to provide crucial insights. With a knack for data, "
    "the Data Analyst Agent is the cornerstone for "
    "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [12]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
    "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
    "markets and quantitative analysis, this agent "
    "devises and refines trading strategies. It evaluates "
    "the performance of different approaches to determine "
    "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [13]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
    "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
    "and logistical details of potential trades. By evaluating "
    "these factors, it provides well-founded suggestions for "
    "when and how trades should be executed to maximize "
    "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [14]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
    "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
    "and market dynamics, this agent scrutinizes the potential "
    "risks of proposed trades. It offers a detailed analysis of "
    "risk exposure and suggests safeguards to ensure that "
    "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

## Creating Tasks

In [15]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [16]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [17]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [18]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [19]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent,
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task,
    ],
    manager_llm=ChatOpenAI(model="gpt-4o", temperature=0.7),
    process=Process.hierarchical,
    verbose=True,
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [20]:
# Example data for kicking off the process
financial_trading_inputs = {
    "stock_selection": "AAPL",
    "initial_capital": "100000",
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Day Trading",
    "news_impact_consideration": True,
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [21]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: In order to continuously monitor and analyze market data for AAPL, identify trends, and predict market movements, I need to delegate tasks to the appropriate team members. I will start by asking the Data Analyst to gather and process the necessary market data for AAPL. 

Action: Delegate work to co-worker
Action Input: {"task": "Gather and process the necessary market data for AAPL.", "context": "We need to continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.", "coworker": "Data Analyst"}


> Entering new CrewAgentExecutor chain...
I need to gather and analyze market data for AAPL using th

- Display the final result as Markdown.

In [22]:
from IPython.display import Markdown

Markdown(result)

A comprehensive risk analysis report detailing potential risks and mitigation recommendations for AAPL:

### Risk Analysis and Mitigation Strategies for AAPL Trading Strategies

#### 1. Earnings Trading Strategy

- **Potential Risks**:
  - **Earnings Surprises**: Unexpected earnings results can lead to significant price volatility, potentially resulting in losses.
  - **Market Sentiment**: Negative market sentiment can adversely affect the stock price even if earnings are positive.
  - **External Factors**: Broader market conditions or geopolitical events can influence stock price movements around earnings reports.

- **Mitigation Strategies**:
  - **Stop-Loss Orders**: Set tight stop-loss orders to limit potential losses in case of adverse price movements.
  - **Diversification**: Avoid concentrating the investment solely in AAPL; diversify across other stocks or sectors.
  - **Hedging**: Use options strategies, such as protective puts, to hedge against potential downside risks.

#### 2. Trend Following Strategy

- **Potential Risks**:
  - **Trend Reversals**: Sudden reversals in the stock price trend can result in losses.
  - **False Signals**: Technical indicators may generate false signals, leading to premature entries or exits.
  - **Market Noise**: Short-term market fluctuations can obscure the underlying trend, causing trading errors.

- **Mitigation Strategies**:
  - **Trailing Stop-Loss Orders**: Implement trailing stop-loss orders to lock in profits and minimize losses if the trend reverses.
  - **Confirmatory Indicators**: Use multiple technical indicators to confirm trends and reduce the likelihood of false signals.
  - **Position Sizing**: Adjust position sizes based on the strength and reliability of the identified trend.

#### 3. Volatility Trading Strategy

- **Potential Risks**:
  - **Implied Volatility**: Changes in implied volatility can affect the pricing of options and the profitability of the strategy.
  - **Market Events**: Unexpected market events can lead to extreme volatility, posing significant risks.
  - **Time Decay**: Options are subject to time decay, which can erode the value of the position over time.

- **Mitigation Strategies**:
  - **Monitor Volatility**: Regularly monitor implied volatility levels and adjust positions accordingly to manage risk.
  - **Diversified Portfolio**: Spread risk by diversifying the portfolio with other stocks or asset classes.
  - **Strategy Adjustment**: Adjust the options strategy (e.g., switch from straddles to strangles) based on market conditions and volatility expectations.

#### 4. Technical Analysis-Based Strategy

- **Potential Risks**:
  - **Over-Reliance on Indicators**: Solely relying on technical indicators without considering fundamental factors can lead to poor trading decisions.
  - **Market Manipulation**: Technical patterns can be manipulated by large market participants, leading to false signals.
  - **Indicator Lag**: Technical indicators often lag behind the actual price movement, resulting in delayed responses.

- **Mitigation Strategies**:
  - **Combine Analysis**: Integrate technical analysis with fundamental analysis to make more informed trading decisions.
  - **Diverse Indicators**: Use a combination of technical indicators to validate signals and reduce the risk of false positives.
  - **Predefined Levels**: Set predefined stop-loss and take-profit levels based on technical analysis to ensure disciplined trading.

#### 5. Event-Driven Strategy

- **Potential Risks**:
  - **Event Outcomes**: The actual outcome of events (e.g., product launches) may differ from market expectations, leading to price volatility.
  - **Rumors and Speculation**: Market rumors and speculation can cause erratic price movements before and after the event.
  - **Timing**: Poor timing of trades around events can result in missed opportunities or losses.

- **Mitigation Strategies**:
  - **Protective Puts**: Use protective puts to hedge against potential negative outcomes from events.
  - **Event Monitoring**: Stay updated on upcoming events and monitor real-time news to make timely trading decisions.
  - **Contingency Plans**: Have contingency plans in place to manage risks associated with unexpected event outcomes.

#### 6. Short-Term Momentum Strategy

- **Potential Risks**:
  - **Rapid Reversals**: Short-term price movements can reverse quickly, leading to potential losses.
  - **Market Sentiment**: Short-term trades are highly sensitive to changes in market sentiment and news.
  - **Execution Risk**: High-frequency trading requires precise execution, and any delays can affect profitability.

- **Mitigation Strategies**:
  - **Tight Stop-Loss Orders**: Use tight stop-loss orders to minimize losses from rapid price reversals.
  - **Real-Time Monitoring**: Continuously monitor market conditions and sentiment to make quick adjustments to positions.
  - **Automated Trading**: Consider using automated trading systems to ensure precise and timely execution of trades.

### Key Considerations for All Strategies
- **Stay Informed**: Continuously monitor market news, analyst ratings, and overall market sentiment to make informed trading decisions.
- **Manage Volatility**: Use appropriate risk management techniques like stop-loss orders, protective puts, and trailing stops to handle volatility.
- **Adapt Strategies**: Be flexible and adapt trading strategies based on evolving market conditions and new information.

By implementing these risk management approaches, traders can better navigate the potential risks associated with each trading strategy for AAPL while aligning with a medium risk tolerance and day trading preferences.